### LLMs with personalities

In [1]:
# imports
import ollama
from dotenv import load_dotenv
import openai
from google import genai
import snscrape.modules.twitter as sntwitter
import pandas as pd

AttributeError: 'FileFinder' object has no attribute 'find_module'

### Celebrities: Elon Musk vs Gordon Ramsay

In [ ]:
# using BeautifulSoup, we will extract data from both celebrities from X.
from bs4 import BeautifulSoup
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
personalities = {}
response = requests.get(headers=headers, url="https://www.personalityclub.com/personality-types/enfj/")
soup = BeautifulSoup(response.text, "html.parser")
title = soup.find("h1", {"class":"entry-title"})
content = soup.find("div", {"class":"entry-content"})
    # print(title.text)
for paragraph in content.find_all("p"):
    if personalities.get("description") is None:
        personalities["description"] = ""
    if getattr(paragraph.span, "strong", None) and paragraph.span.strong.text == "Keirsey Temperament:":
        break
    
    personalities["description"] += paragraph.text + "\n" 

    
def get_personalities():
    return personalities

print(personalities["description"])

class WebScrapper:
    def __init__(self, url):
        self.url = url
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
        self.title = None
        self.personality = {}

    def get_request(self):
        response = requests.get(headers=self.headers, url=self.url)
        return response
    
    def get_content(self):
        response = self.get_content()
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.find("h1", {"class":"entry-title"})
        content = soup.find("div", {"class":"entry-content"})
        for paragraph in content.find_all("p"):
            if personalities.get("description") is None:
                personalities["description"] = ""
            if getattr(paragraph.span, "strong", None) and paragraph.span.strong.text == "Keirsey Temperament:":
                break
            
            personalities["description"] += paragraph.text + "\n"
        return title, personality




Motto:
An inspiration to others (Source)
Description: (Source)
ENFJs seek continuity through harmonious relationships and collective values. They excel at picking up on the tone of a situation and acting accordingly, adding warmth to a cool setting or turning sour into sweet.
They naturally seek to know what people do well, what they enjoy, and where and how they work.

They seem to have an infinite number of acquaintances from all walks of life and are always on the lookout for people in need and those who can help out.
ENFJs weave and strengthen the collective fabric of social conventions and interactions. Inclusiveness is important and they are particularly sensitive to those who are excluded.
ENFJs focus on others, feeling a glow when those around them are happy, and troubled when something is amiss. They are natural cheerleaders, often expressing support, gratitude, and encouragement, and heaping praise onto those they appreciate.
They take note of what is being done and what need